In [11]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import pickle
import torch

In [3]:
# Configuration

HRIZON = ''         # Defines prediction horizon used to search file names. See run.py for naming rules.


## Plot test results (out-of-sample $R^2$)

In [4]:
whole_periods  = pd.DataFrame()

for file in os.listdir('results'):
    if file.startswith(HRIZON) and file.endswith('.csv'):
        weight_lambda = float(file.split(HRIZON+'_')[1].split('.csv')[0])
        path = os.path.join('results', file)
        result = pd.read_csv(path)
        whole_periods[weight_lambda] = result['Whole periods']
        
whole_periods.index = result['Unnamed: 0']
whole_periods[0.000]['Consensus average'] = np.nan

whole_periods

In [ ]:
# Plot out-of-sample R^2 of predicting consensus variables and asset returns for different lambda settings

plt.figure()
whole_periods.iloc[-1].plot(style='o-', color='orange') # Asset Return
whole_periods.iloc[-2].plot(style='s-', color='green')  # Consensus Variablers

plt.grid(visible=True)
plt.xlabel('$\\lambda$')
plt.ylabel('$R^2$')
plt.xticks([0, 0.002, 0.004, 0.006, 0.008, 0.01]) # lambda
plt.legend(['Return', 'Consensus'])

plt.show()
plt.close()

In [ ]:
# Plot out-of-sample R^2 of predicting consensus variables and asset returns for different testing periods

columns = ['2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01']
index_name = ['2013-01-01~2013-12-31','2014-01-01~2014-12-31', '2015-01-01~2015-12-31', '2016-01-01~2016-12-31', '2017-01-01~2017-12-31',
            '2018-01-01~2018-12-31', '2019-01-01~2019-12-31', '2020-01-01~2020-12-31', '2021-01-01~2021-12-31', '2022-01-01~2022-12-31']
i = 0
fig, axs = plt.subplots(nrows=5, ncols=2, sharex=True, sharey=True, figsize=(10, 15))


for file in os.listdir('results'):
    if file.startswith(HRIZON) and file.endswith('.csv'):
        weight_lambda = float(file.split(HRIZON+'_')[1].split('.csv')[0]) # Parse lambda from file name
        path = os.path.join('results', file)
        result = pd.read_csv(path)

        if weight_lambda != 0.0: # Ignore naive feedforward network
            index = np.arange(len(columns))
            ax = fig.axes[i]        
            ax.bar(index-0.4, result.iloc[-1][columns], width=0.4, align='center', label='Return')
            ax.bar(index, result.iloc[-2][columns], width=0.4, align='center', label='Consensus')
            if i == 0:
                ax.legend()
            ax.set_xticks(index-0.3)
            ax.set_xticklabels(tuple(index_name), rotation=45)
            ax.grid(axis="y")
            ax.set_title(f"$\\lambda =$ {str(weight_lambda)}")
            i = i+1

fig.tight_layout()

for ax in fig.get_axes():
    ax.label_outer()

fig.supylabel("$R^2$", size = 15)
plt.tight_layout()

plt.show()
plt.close()

## Evaluate portfolio performance and plot cumulative returns

In [ ]:
# First, load variables required for portfolio performance metrics

# Risk free rates to calculate excess returns

welch_goyal = pd.read_csv('data/raw/welch_goyal_raw.csv')
welch_goyal['yyyymm'] = pd.to_datetime(welch_goyal['yyyymm'], format = '%Y%m')
welch_goyal = welch_goyal.rename(columns={'yyyymm':'date'})
risk_free = welch_goyal[['date', 'Rfree']]
risk_free

In [ ]:
# SNP500 log returns for benchmark

snp_500 = welch_goyal[['date', 'Index']]
snp_500['return'] = np.log(snp_500['Index']) - np.log(snp_500['Index'].shift(1)) # Log return
snp_500.drop('Index', axis=1, inplace=True)
snp_500 = snp_500[(snp_500['date']>=pd.to_datetime('2013-01-01')) & (snp_500['date']<pd.to_datetime('2023-01-01'))].set_index('date')
snp_500

In [ ]:
# Stock price and market cap for value-weighted portfolios

CZD = pd.read_csv('data/raw/open_source_asset_pricing.csv')

stock_info = CZD[['date', 'permno', 'Size', 'Price']]
stock_info['date'] = pd.to_datetime(stock_info['date'])
stock_info

In [ ]:
portfolio_summary = dict()
portfolio_summary['return'] = pd.DataFrame()
portfolio_summary['weight'] = dict()
portfolio_summary['returns'] = dict()

consensus = True  # Choose whether to utilize predicted consensus variables to douvble sort long-short portfolios

color = iter(plt.cm.rainbow(np.linspace(0, 1, 11)))

plt.figure(figsize=(17, 10))

for file in os.listdir('results'):
    if file.startswith('approx') and file.endswith('.pickle'): # Only consider case of 1 month prediction
        weight_lambda = float(file.split('approx_')[1].split('.pickle')[0])
        
        path = os.path.join('results', file)
        with open(file=path, mode='rb') as f:
            output = pickle.load(f)

        actual = output['actual_target']
        actual.columns = ['date', 'permno', 'actual']
        forecast = output['forecast_target']
        forecast.columns = ['date', 'permno', 'forecast']

        returns = pd.merge(actual, forecast).dropna()

        if weight_lambda != 0 and consensus:
            consensuses = output['forecast_concept']
            consensuses.columns = ['date', 'permno', 'EPS forecast revision', 'Change in recommendation', 'Change in Forecast and Accrual', 'Long-vs-short EPS forecasts', 
            'Analyst earnings per share', 'EPS Forecast Dispersion', 'Earnings forecast revisions', 'Analyst Value', 'Analyst Optimism']
            cons_var = 'Analyst earnings per share' # Choose which consensus variable to utilize for double sort
            returns = pd.merge(returns, consensuses[['date', 'permno', cons_var]], how='left')
        returns = pd.merge(returns, stock_info, how='left')

        portfolio_returns = list()
        individual_returns = pd.DataFrame()
        portfolio_weights = pd.DataFrame()

        for snapshot in returns.groupby('date'):
            date = snapshot[0]
            curr_returns = snapshot[1]

            individual_returns = pd.concat([individual_returns, curr_returns[['date', 'permno', 'actual']].set_index('date')])

            if weight_lambda != 0 and consensus:
                # Get return threshold for each positions by percentile
                long_cut  = curr_returns.quantile(0.75)['forecast']
                short_cut = curr_returns.quantile(0.25)['forecast']
                # Get list of firms for each positions
                long_posit  = curr_returns[curr_returns['forecast']>=long_cut ]['permno']
                short_posit = curr_returns[curr_returns['forecast']<=short_cut]['permno']

                # Do the same for consensus variable
                long_con  = curr_returns[curr_returns['permno'].isin(long_posit)].quantile(0.75)[cons_var]
                short_con = curr_returns[curr_returns['permno'].isin(short_posit)].quantile(0.25)[cons_var]
                long_posit  = curr_returns[curr_returns['permno'].isin(long_posit)][curr_returns[cons_var]>=long_con ]['permno']
                short_posit = curr_returns[curr_returns['permno'].isin(short_posit)][curr_returns[cons_var]<=short_con]['permno']
            else:
                # Sort portfolio only once when (consensus == False)
                long_cut  = curr_returns.quantile(0.9)['forecast']
                short_cut = curr_returns.quantile(0.1)['forecast']
                long_posit  = curr_returns[curr_returns['forecast']>=long_cut ]['permno']
                short_posit = curr_returns[curr_returns['forecast']<=short_cut]['permno']
            
            # Get portfolio weights (value-weighted)
            long_total  = curr_returns[curr_returns['permno'].isin(long_posit)]['Size'].sum()
            short_total = curr_returns[curr_returns['permno'].isin(short_posit)]['Size'].sum()
            long_weight  = curr_returns[curr_returns['permno'].isin(long_posit)]['Size']/long_total
            short_weight = -curr_returns[curr_returns['permno'].isin(short_posit)]['Size']/short_total

            portfolio_weight = pd.concat([long_weight, short_weight])
            portfolio_weight.name = 'weight'
            portfolio_weight = curr_returns.join(portfolio_weight, how='left')[['date', 'permno', 'weight']].fillna(0).set_index('date')
            portfolio_weights = pd.concat([portfolio_weights, portfolio_weight])
            
            # Get portfolio returns for each positions
            long_return = curr_returns[curr_returns['permno'].isin(long_posit)]['actual'].values * long_weight
            short_return = curr_returns[curr_returns['permno'].isin(short_posit)]['actual'].values * short_weight

            portfolio_return = pd.concat([long_return, short_return])
            portfolio_returns.append(portfolio_return.sum())

        
        # Plot portfolio cumulative log returns for different lambda settings
        portfolio = pd.DataFrame()
        portfolio.index = returns['date'].unique()
        portfolio['return'] = portfolio_returns
        portfolio['$\\lambda=$'+str(weight_lambda)] = np.exp(portfolio['return']).cumprod()-1
        c = next(color)
        portfolio['$\\lambda=$'+str(weight_lambda)].plot(legend=True, color=c)

        portfolio_summary['return'][weight_lambda]  = portfolio_returns
        portfolio_summary['weight'][weight_lambda]  = portfolio_weights
        portfolio_summary['returns'][weight_lambda] = individual_returns

portfolio_summary['return']['S&P 500'] = snp_500['return'].values
portfolio_summary['return']['date'] = returns['date'].unique()

 # Plot portfolio cumulative log return for s&p500 buy-and-hold strategy as a benchmark
snp_500['S&P 500'] = np.exp(snp_500['return']).cumprod()-1
snp_500['S&P 500'].plot(legend=True, style='--', color='black')

plt.grid()
plt.show()
plt.close()

In [349]:
# Helper function for calculating maximum drawdown
def MAX_DD(data):
    max_dd = -10000

    for t1 in data.index:
        for t2 in data.index:
            if t1<=t2:
                drawdown = data[t1] - data[t2]
                if drawdown > max_dd:
                    max_dd = drawdown
    
    return max_dd

# Helper function to calculate 

def turnover(returns, weights):
    turnovers = list()
    for date1, date2 in zip(returns.index.unique(), returns.index.unique()[1:]):
        w_t_1 = weights.loc[date1]
        w_t_2 = weights.loc[date2]
        r_t = returns.loc[date1]

        permno = set(w_t_1['permno']).intersection(set(w_t_2['permno']))
        w_t_1 = w_t_1[w_t_1['permno'].isin(permno)]['weight']
        w_t_2 = w_t_2[w_t_2['permno'].isin(permno)]['weight']
        r_t = r_t[r_t['permno'].isin(permno)]['actual']

        wr = w_t_1*(1+r_t)
        wr_sum = 1+(r_t*w_t_1).sum()

        turnover = np.abs(w_t_2.values - (wr/wr_sum).values).sum()
        turnovers.append(turnover)
        
    return turnovers

In [ ]:
# Portfolio excess return
portfolio_summary['excess_return'] = portfolio_summary['return'].copy()
portfolio_summary['excess_return'] = pd.merge(portfolio_summary['excess_return'], risk_free, on='date').set_index('date')

for col in portfolio_summary['excess_return'].columns:
    portfolio_summary['excess_return'][col] = portfolio_summary['excess_return'][col] - portfolio_summary['excess_return']['Rfree'] # Calculate excess returns
portfolio_summary['excess_return'].drop('Rfree', axis=1, inplace=True)
portfolio_summary['return'] = portfolio_summary['return'].set_index('date')

In [ ]:
# Calculate portfolio performance evaluation metrics

portfolio_analysis = pd.DataFrame()

# (1) Monthly mean return
portfolio_analysis['mean return'] = portfolio_summary['return'].mean()

# (2) Cumulative log return
cumsum = portfolio_summary['return'].cumsum()
portfolio_analysis['cumulative return'] = cumsum.iloc[-1]

# (3) Annualizedd Sharpe ratio
annualized_return  = portfolio_summary['excess_return'].mean()
annualized_volatility  = portfolio_summary['return'].std()
portfolio_analysis['Sharpe'] = annualized_return *  np.sqrt(12) /annualized_volatility

# (4) Maximum 1 month loss
portfolio_analysis['max 1M loss'] = portfolio_summary['return'].min().abs() * 100

# (5) Maximum Drawdown
portfolio_analysis['max DD'] = cumsum.apply(MAX_DD) * 100

# (6) Portfolio Turnover
portfolio_turnover = pd.Series()

for weight in portfolio_summary['weight']:
    returns = portfolio_summary['returns'][weight]
    weights = portfolio_summary['weight'][weight]

    portfolio_turnover[weight] = np.mean(turnover(returns, weights))

portfolio_analysis['turnover'] = portfolio_turnover * 100
portfolio_analysis

## Analyze predicted consensus variables

In [5]:
results = dict()
consensus_names = ['AnalystRevision',
'ChangeInRecommendation',
'ChForecastAccrual' ,
'EarningsForecastDisparity' ,
'FEPS' ,
'ForecastDispersion' ,
'REV6' ,
'AnalystValue',
'AOP']


for folder in os.listdir('checkpoints'):
    if folder.startswith(HRIZON):
        weight_lambda = float(folder.split(HRIZON+'_')[1])
        path = os.path.join('checkpoints', folder)
        results[weight_lambda] = pd.DataFrame()

        for file in os.listdir(path):
            file_path = os.path.join(path, file)
            weights = torch.load(file_path, map_location=torch.device('cpu')) # Load trained model parameters
            date = file.split('model_')[0]
            
            # Load model weights and biases for return module (Linear regression coefficients of consensus varialbes and asset returns)
            if date in results[weight_lambda].columns:
                results[weight_lambda][date] =  results[weight_lambda][date].add(pd.Series(torch.cat((weights['final_model.weight'][0], weights['final_model.bias']))))
            else:
                results[weight_lambda][date] = torch.cat((weights['final_model.weight'][0], weights['final_model.bias']))

        # Get mean coefficients for each ensemble models
        results[weight_lambda].index = consensus_names + ['bias']    
        results[weight_lambda] = results[weight_lambda]/10
        results[weight_lambda] = results[weight_lambda].mean(axis=1)

results = pd.DataFrame(results)

In [ ]:
i = 0
fig, axs = plt.subplots(nrows=5, ncols=2, sharex=True, sharey=True, figsize=(10, 15))

for col in results.columns:
    if col != 0.0:
        index = np.arange(len(columns))
        ax = fig.axes[i]        
        # Color red and blue for positive and negative coefficients respectively
        ax.bar(index-0.4, results[col], width=0.4, align='center', color=np.where(results[col]>0, 'red', 'blue'))
        ax.set_xticks(index-0.3)
        ax.set_xticklabels(results[col].index, rotation=90)
        ax.grid(axis="y")
        ax.set_title(f"$\\lambda =$ {str(col)}")
        i = i+1
    #results[col].plot.bar(color=np.where(results[col]>0, 'red', 'blue'))
plt.show()
plt.close()

In [ ]:
results.mean(axis=1).plot.barh(color=np.where(results[col]>0, 'red', 'blue'))
plt.grid(axis="x")
plt.gca().invert_yaxis()
plt.show()
plt.close()